# Connect DuckDB with sakila database through the python API

In [1]:
import pandas as pd
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [2]:

# look at the head() of the description dataframe
description.head()

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
3,sakila,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",False
4,sakila,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


# Read everything from the description dataframe

In [3]:
# create a dictionary to keep all all dataframes
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

In [4]:
# check the resulting dictionary
dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

# Register DataFrames to DuckDB

In [5]:
table_names = ['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list',
             'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 
             'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store']

for table_name in table_names:
    duckdb.register(table_name, dfs[table_name])

In [6]:
# show the results of the registration
# Quick overview of all relations
duckdb.sql("DESC;").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",True
1,temp,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, ""NULL"", VARCHAR, BIGINT, VAR...",True
2,temp,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",True
3,temp,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",True
4,temp,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",True
5,temp,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",True
6,temp,main,customer_list,"[ID, name, address, zip_code, phone, city, cou...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",True
7,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
8,temp,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True
9,temp,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True


# Getting to know dataframes using pandas syntax
- Tip: `DESCRIBE` is quickest for a single table; `information_schema.columns` is best for a full catalog view.<br>

- **Help with LLM to getting used to the syntax of Pandas and Duckdb in Python**

In [7]:
# check the content of the df
dfs["film"].head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00


In [8]:

# each value for each key in the dictionary is a pd df
type(dfs["actor"])

pandas.core.frame.DataFrame

In [9]:
# Iterate all tables to preview:
for name in dfs.keys():
    print(f"== {name} ==")
    print(duckdb.sql(f"SELECT * FROM {name} LIMIT 3;").df())

== actor ==
   actor_id first_name last_name         last_update
0       1.0   PENELOPE   GUINESS 2021-03-06 15:51:59
1       2.0       NICK  WAHLBERG 2021-03-06 15:51:59
2       3.0         ED     CHASE 2021-03-06 15:51:59
== address ==
   address_id             address  address2 district  city_id postal_code  \
0           1   47 MySakila Drive      <NA>               300        None   
1           2  28 MySQL Boulevard      <NA>               576        None   
2           3   23 Workhaven Lane      <NA>               300        None   

  phone         last_update  
0       2021-03-06 15:51:54  
1       2021-03-06 15:51:54  
2       2021-03-06 15:51:54  
== category ==
   category_id       name         last_update
0            1     Action 2021-03-06 15:52:00
1            2  Animation 2021-03-06 15:52:00
2            3   Children 2021-03-06 15:52:00
== city ==
   city_id                city  country_id         last_update
0        1  A Corua (La Corua)          87 2021-03-06 15:51:

In [10]:
# All tables in one shot (DuckDB):
duckdb.sql("""
SELECT table_name, column_name, data_type
FROM information_schema.columns     
WHERE table_schema = 'main'
ORDER BY table_name, ordinal_position;
""").df()

,table_name,column_name,data_type
0,actor,actor_id,DOUBLE
1,actor,first_name,VARCHAR
2,actor,last_name,VARCHAR
3,actor,last_update,TIMESTAMP
4,address,address_id,BIGINT
...,...,...,...
115,staff_list,SID,BIGINT
116,store,store_id,BIGINT
117,store,manager_staff_id,BIGINT
118,store,address_id,BIGINT


In [11]:
out = []
for name in dfs.keys():
    df = duckdb.sql(f"DESCRIBE {name};").df()
    df["table_name"] = name
    out.append(df)
pd.concat(out, ignore_index= False)

,column_name,column_type,null,key,default,extra,table_name
0,actor_id,DOUBLE,YES,None,None,None,actor
1,first_name,VARCHAR,YES,None,None,None,actor
2,last_name,VARCHAR,YES,None,None,None,actor
3,last_update,TIMESTAMP,YES,None,None,None,actor
0,address_id,BIGINT,YES,None,None,None,address
...,...,...,...,...,...,...,...
7,SID,BIGINT,YES,None,None,None,staff_list
0,store_id,BIGINT,YES,None,None,None,store
1,manager_staff_id,BIGINT,YES,None,None,None,store
2,address_id,BIGINT,YES,None,None,None,store


# Task 1: EDA with DuckDB and Pandas
Combine duckdb and pandas to do the EDA.<br>
Wrap the raw SQL queries I prepared in a Python triple-quoted string and pass it to DuckDB.

a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [12]:
result_a = duckdb.sql("""
SELECT 
    title,
    length
FROM
    film
WHERE
    length >= 180
ORDER BY
    length DESC
                    """).df()

result_a

,title,length
0,WORST BANGER,185
1,SWEET BROTHERHOOD,185
2,CHICAGO NORTH,185
3,CONTROL ANTHEM,185
4,DARN FORRESTER,185
5,GANGS PRIDE,185
6,SOLDIERS EVOLUTION,185
7,HOME PITY,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


b) Which movies have the word "love" in its title? Show the following columns; title, rating, length, description

Use `ILIKE` instead of `LIKE` for case-insensitive search

In [13]:
result_b = duckdb.sql("""
SELECT
    title,
    rating,
    length,
    description
FROM 
    film
WHERE
    title ILIKE '%love%'; 
""").df()

result_b

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length



In [14]:
result_c = duckdb.sql("""
SELECT
    MIN(length) AS shortest,
    AVG(length) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film;
""").df()

result_c

,shortest,average,median,longest
0,46,115.272,114.0,185


d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie.<br>
The Manager wants to know the **10 most expensive movies to rent per day.**<br>
<br>
**Key points to implement:**
| Step | Objective                                      | Method                                                                            |
|------|------------------------------------------------|-----------------------------------------------------------------------------------|
| 1    | Determine the daily rental cost for each film | Compute `cost_per_day = rental_rate / rental_duration`                           |
| 2    | Identify the highest-cost films               | Sort by `cost_per_day` in descending order and limit results to 10 rows           |
| 3    | Prevent invalid calculations                  | Exclude records where `rental_duration` equals 0                                  |
| 4    | Ensure data quality                           | Filter out records with NULL `rental_rate` or `rental_duration`                  |
| 5    | Maintain consistent ordering                  | Apply secondary sorting, such as by `title`, when values are tied                |


In [15]:
# Phase 1: Check the required tables 
film_data = duckdb.sql("""
SELECT *
FROM film;
                       """).df()
film_data.head(3)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [17]:
rental_data = duckdb.sql("""
SELECT *
FROM rental;
                         """).df()
rental_data.head(3)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2021-03-06 15:53:41
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2021-03-06 15:53:41
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2021-03-06 15:53:41


In [18]:
# Quick check of the columns I need from film table
check_film = duckdb.sql("""
SELECT
  film_id,
  rental_rate,
  rental_duration
FROM film
GROUP BY film_id, rental_rate, rental_duration
ORDER BY rental_rate DESC;
                        """).df()
check_film.head(3)

,film_id,rental_rate,rental_duration
0,575,4.99,3
1,323,4.99,7
2,71,4.99,3
